# Solving mixed integer programs with Julia/JuMP

<i>Copyright 2016, Pedro Belin Castellucci,

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.</i>


In this Notebook, we will explore some basics of the JuMP modeling language. Although it is a modeling language, it enables the usage of different solvers. By ways of an example, we will use [Clp](https://projects.coin-or.org/Clp)
-- a complete list of supported solvers and JuMP documentation can be found at this [link](https://jump.readthedocs.io/en/latest/). Also, JuMP support linear programming, mixed-integer programming, second-order conic programming, semidefinite programming, and nonlinear programming, but we will focus on linear and integer programming here.

The first step is to install JuMP.

In [1]:
Pkg.add("JuMP")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of JuMP
INFO: Use `Pkg.update()` to get the latest versions of your packages


We can also install Clp in the same way.

In [2]:
Pkg.add("Clp");

INFO: Cloning cache of Cbc from https://github.com/JuliaOpt/Cbc.jl.git
INFO: Cloning cache of Clp from https://github.com/JuliaOpt/Clp.jl.git
INFO: Installing Cbc v0.2.4
INFO: Installing Clp v0.2.2
INFO: Building Cbc
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of Clp
INFO: Use `Pkg.update()` to get the latest versions of your packages


To get a feeling of JuMP basic function we will implement the following model:

$Max\ x + y$

subject to:

$2x + 4y \leq 10,$

$2x + 2y \leq 10.$

We begin by creating a Model object.

In [3]:
using JuMP  # Tell that we are using the JuMP library
using Clp   # Tell that we are using the Clp solver

m = Model()

INFO: Recompiling stale cache file /home/juser/.julia/lib/v0.5/DataStructures.ji for module DataStructures.
INFO: Recompiling stale cache file /home/juser/.julia/lib/v0.5/JuMP.ji for module JuMP.
INFO: Precompiling module Clp.


Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

The second step is to define the variables. Their are associated with the model and can created using the <i>@variable</i> macro.

In [4]:
@variable(m, x)
@variable(m, y)

y

Then, the objective function.

In [5]:
@objective(m, Max, x + y)

x + y

Finally, the constraints.

In [6]:
@constraint(m, 2x + 3y <= 10)
@constraint(m, 3x + 2y <= 10)

m  # Just to diplay our model

Maximization problem with:
 * 2 linear constraints
 * 2 variables
Solver is default solver

Now that we have completely defined our model, we may want to solve it.

In [7]:
solve(m)

:Optimal

The solve function returns the status of the solving procedure. We can query for the solution and variable values using <i>getobjectivevalue</i> and <i>getvalue</i> functions.

In [9]:
println("Objective function = ", getobjectivevalue(m))

println("x = ", getvalue(x))
println("y = ", getvalue(y))

Objective function = NaN
x = NaN
y = NaN


## Exercise

Implement and solve the following model:

$Min\ 2x - 3y + z$

subject to:

$x - y - z = 10,$

$3x - 4y + 4z \geq 25,$

$x, y, z \geq 0.$

## The least absolute method

In this example, let us assume we have some experimental data and want to fit a linear model to it. Perhaps, the most famous method to accomplish this is to use the Least Square Method, which finds the linear model that minimizes the squared error, i. e. the distance from data points to the model. Instead of minimizing the square error, we will minimize the absolute error. More formally, we have experimental points $(x_i, y_i)$, $i = 1, \ldots, n$ and we want the parameters $(a, b)$ of a function $f(x) = ax + b$ such that:

$$\sum_{i=1}^n |f(x_i) - y_i|$$

is minimized. We can formulate a linear programming model of the problem using $a$ and $b$ as decision variables:

$\displaystyle Min\ \sum_{i=1}^n\epsilon_i$,

subject to:

$\displaystyle y_i - a x_i - b \leq \epsilon_i, \quad i=1, \ldots, n$,

$\displaystyle - (y_i - a x_i - b) \leq \epsilon_i, \quad i=1, \ldots, n$ .

To test it, let us generate some data.

In [10]:
n = 10  # We will use 10 points

points = Array{Float64, Float64}[]
points = zeros(0, 2)

println(points)
for i=1:n
    
    # They are generated around the curve f(x) = x
    xRand = rand()*3 - 1
    yRand = rand()*3 - 1

    x, y = i + xRand, i + yRand
    points = [points; x y]
end

points

10×2 Array{Float64,2}:
  0.66462   1.39885
  3.35755   2.55885
  2.7481    2.23233
  5.62835   5.95145
  4.26772   5.46657
  7.07984   6.48674
  6.73943   7.09982
  7.13616   9.67772
 10.0345    8.73821
  9.2365   10.7758 

To get a sense of the data, let us scatter plot it. For that, we are going to use the [Plots library](https://juliaplots.github.io).

In [11]:
Pkg.add("Plots")

INFO: Nothing to be done


And then, we do a scatter plot of the points.

In [29]:
using Plots
plotly()  # The backend we are using

plot(points[:, 1], points[:, 2], t=[:scatter], label="", xlabel='x', ylabel='y')


Now, we implement the model presented earlier.

In [33]:
using JuMP

m = Model()  # Creating the model object

# Defining the variables:
@variable(m, err[1:n])  # error (epsilon)
@variable(m, a)  # angular coefficient
@variable(m, b)  # linear coefficient

@objective(m, Min, sum(err[i] for i in 1:n))

for i in 1:n
    @constraint(m, points[i, 2] - a*points[i, 1] - b <= err[i])
    @constraint(m, - (points[i, 2] - a*points[i, 1] - b) <= err[i])
end

m

Minimization problem with:
 * 20 linear constraints
 * 12 variables
Solver is default solver

We want to solve the model and get the values of $a$ and $b$.

In [34]:
println("We have obtained an $(solve(m)) solution.")
println("a = $(getvalue(a))")
println("b = $(getvalue(b))")

We have obtained an Optimal solution.
a = 0.9171724779882611
b = 0.7892773708643734


Let us plot the solution to see the result.

In [39]:
fig = plot(points[:, 1], points[:, 2], t=[:scatter], label="")

y = [getvalue(a)*points[i, 1] + getvalue(b) for i in 1:size(points,1)]
plot!(points[:, 1], y, w=4, xlabel="x", ylabel="y", label="")

fig

Out of curiosity, we can compare our result with a standard Least Square regression. For this, we can use the GLM and DataFrames packages.

In [31]:
Pkg.add("GLM")
Pkg.add("DataFrames")

INFO: Cloning cache of Distributions from https://github.com/JuliaStats/Distributions.jl.git
INFO: Cloning cache of GLM from https://github.com/JuliaStats/GLM.jl.git
INFO: Cloning cache of PDMats from https://github.com/JuliaStats/PDMats.jl.git
INFO: Cloning cache of Rmath from https://github.com/JuliaStats/Rmath.jl.git
INFO: Cloning cache of StatsBase from https://github.com/JuliaStats/StatsBase.jl.git
INFO: Cloning cache of StatsFuns from https://github.com/JuliaStats/StatsFuns.jl.git
INFO: Installing Distributions v0.11.1
INFO: Installing GLM v0.6.1
INFO: Installing PDMats v0.5.2
INFO: Installing Rmath v0.1.6
INFO: Installing StatsBase v0.12.0
INFO: Installing StatsFuns v0.3.1
INFO: Building Rmath
INFO: Package database updated
INFO: Cloning cache of DataArrays from https://github.com/JuliaStats/DataArrays.jl.git
INFO: Cloning cache of DataFrames from https://github.com/JuliaStats/DataFrames.jl.git
INFO: Cloning cache of FileIO from https://github.com/JuliaIO/FileIO.jl.git
INFO: Clo

In [42]:
using DataFrames
using GLM
data = DataFrame(X=points[:, 1], Y=points[:, 2])
result = glm(Y ~ X, data, Normal(), IdentityLink())
bLS, aLS = coef(result) 

fig = plot(points[:, 1], points[:, 2], t=[:scatter], label="")

y = [getvalue(a)*points[i, 1] + getvalue(b) for i in 1:size(points,1)]
plot!(points[:, 1], y, label="Ours", w=3)

y = [aLS*points[i, 1] + bLS for i in 1:size(points,1)]
plot!(points[:, 1], y, label="LSM", w=3)

fig

We can see the curves are similar. The least square method have the characteristic of heavily penalize outliers, so point to far from the mean may have a big influence on regression. Note that we have generated the points randomly, you can rerun the experiments to see possible differences in the results.

## A more classical example -- The knapsack problem

Let $i \in I$ be an item with value $v_i$ and volume $c_i$. We want to choose the most valuable subset of items to carry in a knaspack withouth violating its capacity $C$. Let $x_i \in \{0, 1\}$, $i \in I$ indicate whether item $i$ is put into the knapsack. The following integer program solver our knapsack problem.

$\displaystyle Max\ \sum_{i \in I} v_i x_i$

subject to

$\displaystyle \sum_{i \in I} c_i x_i \leq C,$

$x_i \in \{0, 1\}, i \in I.$

We will implement this program using JuMP. First, we need some input data.

In [ ]:
v = [3, 1, 4, 6, 2, 7, 2, 4, 7, 9, 5, 7, 3, 1]  # Value of the items.
c = [2, 5, 3, 8, 9, 6, 4, 8, 2, 2, 1, 4, 2, 3]  # Volume of the items.
n = length(v)  # How many items we have

C = 12  # The capacity of our knapsack.

We do the same as before, create our model.

In [11]:
m = Model()

Feasibility problem with:
 * 0 linear constraints
 * 0 variables
Solver is default solver

But now, we are not going to creating the variables one by one, instead we will use the following.

In [12]:
@variable(m, x[i=1:n], Bin)

10-element Array{JuMP.Variable,1}:
 x[1] 
 x[2] 
 x[3] 
 x[4] 
 x[5] 
 x[6] 
 x[7] 
 x[8] 
 x[9] 
 x[10]

Which is a shortcut for adding the variables <i>x[1]</i> to <i>x[|n|]</i> to the model <i>m</i>. To add the objective function we can use the <i> sum </i> function.

In [ ]:
@objective(m, Max, sum(v[i]*x[i] for i=1:n))

It can also be used to create constraints.

In [ ]:
@constraint(m, sum(x[i]*c[i] for i=1:n) <= C)

m  # Just to print our model.

And then, we can solve our model.

In [ ]:
solve(m)

Finally, we can query for the solution.

In [ ]:
println("Take \t Don't take")

for i in collect(1:n)
    if getvalue(x[i]) >= 0.99
        println("$i")
    else
        println("\t $i")
    end
end

In this Notebook, we have explored the basics of JuMP regarding mixed integer linear programming. For a more detailed exposition the reader is referred to [JuMP documentation](https://jump.readthedocs.io/en/latest/).

Hope you had fun! (=